In [ ]:
# Avg Length of tweets	
# Types of Special chars, non ascii chars	
# Types of special chars for each category	
# Total no of tweets by category	
# Avg Length of tweets per category	
# Upper case words present in each category	
# Typo vs abbreviations in each category	
# Presence of disaster words within text for labeled with disaster	
# Presence of exclamatory words	
# Presence of URLs and significance	
# Mention to external entities/places/govt agencies in the text	
# Extra line breaks,spaces	
# Data issues like line splits for single record	
# Elimination of data that is not labeled	
# Identify Retweet using urls	
# Analyze the data that has missing values	


In [ ]:
!pip install urllib3
!pip install transformers
!pip install -q tf-models-official
!pip install seaborn 
!pip install tensorflow_text
!pip install opencv-python-headless
!pip install clean-text
!pip install demoji
!pip install tensorflow-gpu 

In [ ]:
!pip install contractions
!pip install spellchecker

In [ ]:
!ls ../input/nlp-getting-started

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#cleaning text
import re
import unidecode
from nltk.stem import PorterStemmer, WordNetLemmatizer
import emoji
import contractions
from textblob import TextBlob
#from spellchecker import SpellChecker
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


#named entity recognition
import spacy
import en_core_web_sm

#wordcloud 
from collections import Counter
from wordcloud import WordCloud

#tokenizing 
from nltk.tokenize import TweetTokenizer

#import chart_studio.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
import plotly.express as px
import plotly.graph_objs as go
import demoji

In [ ]:
df = pd.read_csv('../input/nlp-getting-started/train.csv',delimiter=',',encoding='latin-1')
df.head()

In [ ]:
#summary of dataframe
df.info()

In [ ]:
dfdata = pd.read_csv('../input/nlp-getting-started/train.csv',delimiter=',',encoding='latin-1')
dfdata.head()

In [ ]:
df.shape

In [ ]:
ax = dfdata.groupby('target').count().plot(kind='bar',legend=True)
plt.show();

In [ ]:
print('Percentage of missing data')
print((dfdata.isna().sum()/len(dfdata))*100)
print(dfdata['id'].count())
print(dfdata.isnull().sum())
 

In [ ]:
keywords = dfdata.drop(['location', 'text'], axis = 1)
keywords.shape

In [ ]:
keywords = keywords.dropna()

In [ ]:
print(keywords.nunique())

In [ ]:
keywords.shape

In [ ]:
def clean_keywords(keyword):
    cleaned = re.sub(r'%20', ' ', keyword)
    return cleaned
def remove_accents(keyword):
    cleaned = unidecode.unidecode(keyword)
    return cleaned
def remove_punctuation(keyword):
    cleaned = re.sub(r"[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n -' ]"," ",keyword)
    return cleaned
keywords['keyword'] = keywords['keyword'].apply(lambda x: clean_keywords(x))
keywords['keyword'] = keywords['keyword'].apply(lambda x: remove_accents(x))
keywords['keyword'] = keywords['keyword'].apply(lambda x: remove_punctuation(x))

In [ ]:
stemmer = PorterStemmer()
keywords['stems'] = keywords['keyword'].apply(lambda x: stemmer.stem(x))

In [ ]:
keywords.head(2)

In [ ]:
keywords['target_mean_stem'] = keywords.groupby('stems')['target'].transform('mean')
fig = plt.figure(figsize=(8, 40), dpi=100)
sns.countplot(y=keywords.sort_values(by='target_mean_stem', ascending = False)['stems'], 
              hue = keywords.sort_values(by='target_mean_stem', ascending =False)['target'])
plt.show()

In [ ]:
keywords['keywordcnt'] = keywords.groupby('keyword')['target'].transform('mean')
fig = plt.figure(figsize=(8, 40), dpi=100)
sns.countplot(y=keywords.sort_values(by='keywordcnt', ascending = False)['keyword'], 
              hue = keywords.sort_values(by='keywordcnt', ascending =False)['target'])
plt.show()

In [ ]:
def make_dict(tup, dictionary):
    for x, y in tup:
        dictionary.setdefault(x, []). append(y)
    return dictionary

In [ ]:
#Location
location_disaster = dfdata[dfdata['target']==1].drop(['text','keyword'], axis = 1)
location_nondisaster = dfdata[dfdata['target']==0].drop(['text','keyword', 'id'], axis = 1)
print('Missing data for Disaster Tweets')
print((location_disaster.isnull().sum()/len(location_disaster))*100)
print('\n')
print('Missing data for Non Disaster Tweets')
print((location_nondisaster.isnull().sum()/len(location_nondisaster))*100)

In [ ]:
location = dfdata[dfdata['location'].notnull()].drop(['text', 'keyword'], axis = 1)
location.head(2)
location

In [ ]:
print(location['location'].nunique())
print(len(location))

In [ ]:
location.info

In [ ]:
location['location'] = location['location'].apply(lambda x: remove_accents(x))
location['location'] = location['location'].apply(lambda x: remove_punctuation(x))

In [ ]:
#let's remove numerical entries
def remove_nums(location):
    cleaned = re.sub(r'\d+', '', location)
    return cleaned
location['location'] = location['location'].apply(lambda x: remove_nums(x))
def remove_extra_w_space(location):
    cleaned_text = re.sub(r"\s+"," ",location).strip()
    return cleaned_text
location['location'] = location['location'].apply(lambda x: remove_extra_w_space(x))

In [ ]:
location['location'].nunique()

In [ ]:
location_disaster = location[location['target']==1]
location_nondisaster = location[location['target']==0]

In [ ]:
nlp = en_core_web_sm.load()
tokens = nlp(''.join(str(location_disaster.location.tolist())))
items = [word.text for word in tokens.ents]
disasters = Counter(items).most_common(3000)

In [ ]:
nlp = en_core_web_sm.load()
tokens = nlp(''.join(str(location_nondisaster.location.tolist())))
items = [word.text for word in tokens.ents]
non_disasters = Counter(items).most_common(3000)

In [ ]:
x = list(set(disasters).intersection(non_disasters))
len(x)

In [ ]:
tokens = nlp(''.join(str(location_disaster.location.tolist())))
items = [word.text for word in tokens.ents]
disasters = Counter(items).most_common(3106)
nlp = en_core_web_sm.load()
tokens = nlp(''.join(str(location_nondisaster.location.tolist())))
items = [word.text for word in tokens.ents]
non_disasters = Counter(items).most_common(3106)

In [ ]:
df1 = pd.DataFrame(disasters, columns=['disaster_words', 'count'])

In [ ]:
df2  = pd.DataFrame(non_disasters,columns=['non_disaster_words', 'counts'] )

In [ ]:
location_words = pd.concat([df1, df2])
location_words = location_words.drop(['count', 'counts'], axis = 1)

In [ ]:
disaster_words = location_words.disaster_words.dropna().map(' '.join)
non_disaster_words = location_words.non_disaster_words.dropna().map(' '.join)
corpus = [disaster_words, non_disaster_words]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
vectorizer= MultiLabelBinarizer()
mlb = vectorizer.fit(corpus)

dfloc = pd.DataFrame(mlb.transform(corpus), columns=[mlb.classes_], index=['Disaster', 'Non Disaster'])

In [ ]:
#text
text = dfdata.drop(['keyword', 'location'], axis = 1)

In [ ]:
#Hashtags, Mentions, and Emojis
def hash_count(string):
    words = string.split()
    hashtags = [word for word in words if word.startswith('#')]
    return len(hashtags)
text['num_hashtags'] = text['text'].apply(hash_count)
text['hashtags'] = text['text'].apply(lambda x: [x for x in x.split(" ") if x.startswith("#")])
def mention_count(string):
    words = string.split()
    mentions = [word for word in words if word.startswith('@')]
    return len(mentions)
text['num_mentions'] = text['text'].apply(mention_count)
text['mentions'] = text['text'].apply(lambda x: [x for x in x.split(" ") if x.startswith("@")])
def emoji_count(tweet):
    tweet = emoji.demojize(tweet, delimiters=('__','__'))
    pattern = r'_+[a-z_&]+_+'
    return len(re.findall(pattern, tweet))
text['emojis'] = text['text'].apply(emoji_count)

In [ ]:
disaster_tweets = text[text['target']==1]
nondisaster_tweets = text[text['target']==0]

In [ ]:
print(max(disaster_tweets['text'].str.count(r'[A-Z]')))

In [ ]:
text[text['num_hashtags']>=11]

In [ ]:
def expand_contractions(tweet):
    return contractions.fix(tweet)
text['text'] = text['text'].apply(lambda x: expand_contractions(x))

In [ ]:
def demojize(tweet):
    emojis = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags = re.UNICODE)
    cleaned_text = emojis.sub(r'', tweet)
    return cleaned_text
text['text'] = text['text'].apply(lambda x: demojize(x))

def emojis(tweet):
    tweet = emoji.demojize(tweet)
    tweet = tweet.replace(':','')
    cleaned_text = ' '.join(tweet.split())
    return cleaned_text

In [ ]:
text['text'] = text['text'].apply(lambda x: remove_accents(x))
text['text'] = text['text'].apply(lambda x: remove_punctuation(x))

In [ ]:
#now let's remove the hashtags, mentions, html text, numbers, and urls 
def remove_hashtag(tweet):
    cleaned_text = re.sub(r'#\w+','',tweet)
    return cleaned_text
text['text'] = text['text'].apply(lambda x: remove_hashtag(x))
def remove_mentions(tweet):
    cleaned_text = re.sub(r'@\w+|@[^\s]+','',tweet)
    return cleaned_text
text['text'] = text['text'].apply(lambda x: remove_mentions(x))
def remove_html_text(tweet):
    html_text = re.compile(r'<.*?>|\n|\t|\&\w*;z|\$\w*')
    cleaned_text = html_text.sub(r'',tweet)
    return cleaned_text
text['text'] = text['text'].apply(lambda x: remove_html_text(x))
def remove_url_text(tweet):
    url = re.compile(r'https?://\S+|www\.\S+|http://t.co/|http|https?:\/\/.*\/\w*')
    cleaned_text = url.sub(r'',tweet)
    return cleaned_text
text['text'] = text['text'].apply(lambda x: remove_url_text(x))

In [ ]:
#remove numerical values 
def remove_nums(tweet):
    cleaned_text = re.sub(r'\d+', '', tweet)
    return cleaned_text
text['text'] = text['text'].apply(lambda x: remove_nums(x))
#change all tweets to lower case
def lower_case(tweet):
    lower_case = tweet.lower()
    return lower_case
text['text'] = text['text'].apply(lambda x: lower_case(x))

In [ ]:
def remove_extra_w_space(tweet):
    cleaned_text = re.sub(r"\s+"," ",tweet).strip()
    return cleaned_text
text['text'] = text['text'].apply(lambda x: remove_extra_w_space(x) )
#lets also remove any white space before the tweets
text['text'] = text['text'].apply(lambda x: x.lstrip(' '))

In [ ]:
text.head()

In [ ]:
dfnew = pd.merge(text, location, on="id")

In [ ]:
dfnew= pd.merge(dfnew, keywords, on="id")

In [ ]:
cols = dfnew.columns
dfnew['combined'] = dfnew['text'] # + ' ' + dfnew['location'] + ' ' + dfnew['keyword']

In [ ]:
dfnew['text'] = dfnew['combined']

In [ ]:
dfnew1 = dfnew[['text','target']]

In [ ]:
#dfnew1.to_csv('train2.csv')

In [ ]:
dfnew1['target'].isnull().values.any()

In [ ]:
dfnew1.target.unique() 

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dfnew1.text,\
                                                    dfnew1.target,\
                                                    test_size=0.10)

In [ ]:
# bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
import tensorflow as tf
def get_BERT_model():
    # Preprocessing
    tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
    # Bert encoder    
    #fhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2'
    tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
    bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
    bert_model = hub.KerasLayer(tfhub_handle_encoder)
    input_layer = tf.keras.layers.Input(shape=(), dtype=tf.string, name='tweets')
    x = bert_preprocess_model(input_layer)
    x = bert_model(x)['pooled_output']
    x = tf.keras.layers.Dropout(0.5)(x) #Optional, to eliminate overfitting
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    classification_out = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(x)
    bert_preprocess_model._name = "preprocess"
    bert_model._name = "bert_encoder"
    modelbert = tf.keras.Model(input_layer, classification_out)
    modelbert._name = "modelbert"
    return modelbert
bertmodel = get_BERT_model()
bertmodel.summary()

In [ ]:
import tensorflow as tf
from official.nlp import optimization
#from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# class_weights = compute_class_weight(class_weight = "balanced", 
#                                      classes = np.unique(dfnew1["target"]),
#                                      y= dfnew1["target"])
# class_weights = {k:class_weights[k] for k in np.unique(dfnew1["target"])}
# class_weights

In [ ]:
epochs = 10
steps_per_epoch = X_train.count()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)
init_lr = 5e-5
#optimizer=tf.keras.optimizers.Adam(learning_rate=init_lr )
optimizer = tf.keras.optimizers.AdamW(learning_rate=init_lr) 
optimizer = tfa.optimizers.LAMB(learning_rate=init_lr) 
#optimizer='SGD'              
#optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3,momentum=0.9,nesterov=False)

In [ ]:
# loss = tf.keras.losses.BinaryFocalCrossentropy(  alpha=0.25,
#                                                  gamma=2.0 ,
#                                                apply_class_balancing=True
#                                               )
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
#loss='binary_crossentropy'

In [ ]:
Benchmarks = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')     
]

In [ ]:
bertmodel.compile(optimizer=optimizer,
              loss=loss,
              metrics= Benchmarks)

In [ ]:
tf.keras.utils.plot_model(bertmodel)

In [ ]:
bertmodel.get_layer('bert_encoder').trainable = False

In [ ]:
out = bertmodel.fit(X_train, y_train, epochs=epochs)

In [ ]:
def plot_hist(hist):
    '''
    Plots the training / validation loss and accuracy given the training history
    '''
    category = ["recall","loss","accuracy"]
    for item in category:
        plt.plot(hist.history[item])       
        plt.ylabel(plt)
        plt.xlabel("Epoch")
        plt.legend([item])
        plt.grid()
        plt.show()
 
plot_hist(out)

In [ ]:
# import joblib
# joblib_file = "joblib_bert_Model2.pkl"  
# joblib.dump(bertmodel, joblib_file)

In [ ]:
rslt = bertmodel.evaluate(X_test,y_test)

In [ ]:
def get_prediction(df,model):
    """
    """
    predicted = model.predict(df)
    predicted  = predicted.flatten()
    return np.where(predicted>0.5,1,0)


In [ ]:
p1 = bertmodel.predict(X_test[:]) 
p1  = p1.flatten()
print(p1)
print(np.where(p1>=0.5,1,0))

In [ ]:
predicted = get_prediction(X_test,bertmodel)
print(predicted)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
con_matr = confusion_matrix(y_test, predicted)
sn.heatmap(con_matr,annot=True,fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Actual')


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted))

In [ ]:
#unseen data
#hyper parameter tuning


https://sandipanweb.wordpress.com/2023/03/31/nlp-disaster-tweets-kaggle-mini-project/
https://snrspeaks.medium.com/fine-tuning-xlnet-model-for-text-classification-in-3-lines-of-code-1a7c3b320669
https://www.kaggle.com/code/donkeys/distilbert-xlnet-with-tf-and-huggingface
https://www.kaggle.com/code/kyakovlev/preprocessing-bert-public